In [ ]:
# for colab import
#!pip install -q xlrd
#!git clone https://github.com/onimaru/CursoBio.git

In [ ]:
#!ls CursoBio/datasets/newHIV-1_data/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
data_746_raw = pd.read_csv(r".\datasets\newHIV-1_data\746Data.txt", header=None)
data_1625_raw = pd.read_csv(r".\datasets\newHIV-1_data\1625Data.txt", header=None)
data_impens_raw = pd.read_csv(r".\datasets\newHIV-1_data\impensData.txt", header=None)
data_schilling_raw = pd.read_csv(r".\datasets\newHIV-1_data\schillingData.txt", header=None)

In [ ]:
data_7 = data_746_raw.copy()

In [ ]:
data_1 = data_1625_raw.copy()

In [ ]:
data_i = data_impens_raw.copy()

In [ ]:
data_s = data_schilling_raw.copy()

In [ ]:
data_s.head()

In [ ]:
data_s.columns = ['Peptide','Cleavage']
print(data_s.shape)
data_s.head()

In [ ]:
data_s['Prot_sum'] = data_s['Peptide'].apply(len)
print('Número máximo:',max(data_s['Prot_sum']))
print('Número mínimo:',min(data_s['Prot_sum']))
print(data_s.head())

In [ ]:
# Separando a string em Peptide em uma coluna para cada proteína
n = max(data_s['Peptide'].apply(len))
for i in range(n):
    data_s['Pep0'+str(i)] = data_s['Peptide'].str[i]
data_s.head()

In [ ]:
# target feature
y_s = pd.DataFrame(data_s['Cleavage'])
y_s = y_s.replace(-1,0)
print(y_s.shape)
y_s.head()

In [ ]:
# X features
X_s = data_s.drop(['Peptide','Cleavage','Prot_sum'],axis=1)
print(X_s.shape)
X_s.head()

In [ ]:
X_s_enc = pd.get_dummies(X_s)
print(X_s_enc.shape)
X_s_enc.head()

In [ ]:
features = X_s_enc.columns

In [ ]:
# preparing other datasets
data_7.columns = ['Peptide','Cleavage']
n = max(data_7['Peptide'].apply(len))
for i in range(n):
    data_7['Pep0'+str(i)] = data_7['Peptide'].str[i]
y_7 = pd.DataFrame(data_7['Cleavage'])
y_7 = y_7.replace(-1,0)
X_7 = data_7.drop(['Peptide','Cleavage'],axis=1)
X_7_enc = pd.get_dummies(X_7)
print(X_7_enc.shape)
print(y_7.shape)
X_7_enc.head()

In [ ]:
X = X_s_enc.values
y = y_s.values

In [ ]:
X_test = X_7_enc.values
y_test = y_7.values

In [ ]:
# save files
X_s_enc.to_csv(r'.\datasets\newHIV-1_data\X_s.csv')
y_s.to_csv(r'.\datasets\newHIV-1_data\y_s.csv')

X_7_enc.to_csv(r'.\datasets\newHIV-1_data\X_7.csv')
y_7.to_csv(r'.\datasets\newHIV-1_data\y_7.csv')

***Training phase***

In [ ]:
# machine learning libraries
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_predict, GridSearchCV, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.cross_validation import KFold

# metrics
from sklearn.metrics import accuracy_score, log_loss, make_scorer, confusion_matrix, f1_score, precision_score,\
        recall_score, precision_recall_curve, roc_curve, roc_auc_score

# aditional libraries
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def classifier_scores(true_labels,prediction_labels):
    f1 = f1_score(true_labels,prediction_labels)
    pre = precision_score(true_labels,prediction_labels)
    rec = recall_score(true_labels,prediction_labels)
    acc = accuracy_score(true_labels,prediction_labels)
    auc = roc_auc_score(true_labels,prediction_labels)
    report = pd.DataFrame({'AUC':np.around([auc],3),'Precision':np.around([pre],3), 'Recall':np.around([rec],3),'F1':np.around([f1],3),'Accuracy':np.around([acc],3)})
    print(report)

In [ ]:
#  list of classifiers
classifiers = [
    KNeighborsClassifier(5),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    SGDClassifier(),
    LogisticRegression()]

In [ ]:
# graph to compare different regressors with the same chosen metric
# classifiers = list of classifiers we want to use
# cv = number of desired folds
def scoringGraph(classifiers,cv,X,y):
  init = time.time()
  
  log_cols = ["Classifier", "F1_score"]
  log 	 = pd.DataFrame(columns=log_cols)
  splits = 5
  sss = StratifiedShuffleSplit(n_splits=splits, test_size=0.1, random_state=0)

  acc_dict = {}

  for train_index, test_index in sss.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for clf in classifiers:
      name = clf.__class__.__name__
      clf.fit(X_train, y_train)
      y_pred = clf.predict(X_test)
      acc = f1_score(y_test, y_pred)
      if name in acc_dict:
        acc_dict[name] += acc
      else:
        acc_dict[name] = acc

  for clf in acc_dict:
    acc_dict[clf] = acc_dict[clf] / splits
    log_entry = pd.DataFrame([[clf, acc_dict[clf]]], columns=log_cols)
    log = log.append(log_entry)

  plt.xlabel('F1_score')
  plt.title('Classifier F1_score')

  sns.set_color_codes("muted")
  sns.barplot(x='F1_score', y='Classifier', data=log, color="b");

  print('Tempo total: {:.2f}'.format(time.time()-init))

In [ ]:
scoringGraph(classifiers,4,X,y)

***Melhorando os modelos***

In [ ]:
clf = DecisionTreeClassifier()

In [ ]:
scorer = make_scorer(roc_auc_score)
score = cross_val_score(clf,X,y,cv=5,scoring=scorer)
score

In [ ]:
y_pred = cross_val_predict(clf,X,y,cv=5)
classifier_scores(y,y_pred)

In [ ]:
plot_confusion_matrix(cm_sgd,classes=['Negative cleavage','Positive cleavage'],normalize=False,title='Not-Normalized confusion matrix')

In [ ]:
y_scores = cross_val_predict(clf,X, y,cv=5)

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y,y_pred)

In [ ]:
#Plot precision and recall as functions of the threshold value
def  plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
plt.figure(figsize=(8, 4))
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)

In [ ]:
plt.plot(precisions,recalls)
plt.xlabel("Precision", fontsize=16)
plt.ylabel("Recall", fontsize=16)

In [ ]:
fpr, tpr, thresholds = roc_curve(y,y_scores)
print('AUC: {:.2f}'.format(roc_auc_score(y,y_scores)))

def  plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

plt.figure(figsize=(8, 6))
plot_roc_curve(fpr, tpr)

## GridSerarchCV

**DecisionTreeClassifier**

In [ ]:
classifier = DecisionTreeClassifier()

# escolhemos alguns parâmetros para a procura
parameters = {'criterion':['gini','entropy'],'min_samples_split':[3,7,10],'max_features':['auto','sqrt','log2',None]}

# escolhemos uma métrica
scorer = make_scorer(roc_auc_score)

# rodamos o grid search no training set
grid_obj = GridSearchCV(classifier, parameters, scoring=scorer)
grid_obj = grid_obj.fit(X, y)

# definimos o classificador knn com os melhores parâmetros
classifier = grid_obj.best_estimator_
best_dt = grid_obj.best_estimator_
# e então treinamos o algoritmo com esta combinação
classifier.fit(X, y)

print(classifier.__class__.__name__)
print('Train set results:')
y_pred_train = classifier.predict(X)
classifier_scores(y,y_pred_train)
print('')
print('Test set results:')
y_pred_test = classifier.predict(X_test)
classifier_scores(y_test,y_pred_test)
print('')
# aditional info
feat = classifier.feature_importances_
print('Max feature importance: {:.2f}'.format(max(feat)))

***RandomForestClassifier***

In [ ]:
classifier = RandomForestClassifier()

# escolhemos alguns parâmetros para a procura
parameters = {'n_estimators':[10,15,20],'criterion':['gini','entropy'],'min_samples_split':[3,7,10],'max_features':['auto','sqrt','log2',None]}

scorer = make_scorer(roc_auc_score)
grid_obj = GridSearchCV(classifier, parameters, scoring=scorer)
grid_obj = grid_obj.fit(X, y)
classifier = grid_obj.best_estimator_
classifier.fit(X, y)

print(classifier.__class__.__name__)
print('Train set results:')
y_pred_train = classifier.predict(X)
classifier_scores(y,y_pred_train)
print('')
print('Test set results:')
y_pred_test = classifier.predict(X_test)
classifier_scores(y_test,y_pred_test)
print('')
# aditional info
feat = classifier.feature_importances_
print('Max feature importance: {:.2f}'.format(max(feat)))

**SVC**

In [ ]:
classifier = SVC()

# escolhemos alguns parâmetros para a procura
parameters = {'C':[0.001,0.01,1.,10.],'kernel':['linear','poly','rbf','sigmoid'],'degree':[1,2,3,4]}

scorer = make_scorer(roc_auc_score)
grid_obj = GridSearchCV(classifier, parameters, scoring=scorer)
grid_obj = grid_obj.fit(X, y)
classifier = grid_obj.best_estimator_
classifier.fit(X, y)

print(classifier.__class__.__name__)
print('Train set results:')
y_pred_train = classifier.predict(X)
classifier_scores(y,y_pred_train)
print('')
print('Test set results:')
y_pred_test = classifier.predict(X_test)
classifier_scores(y_test,y_pred_test)
print('')

**AdaBoostClassifier**

In [ ]:
classifier = AdaBoostClassifier()

# escolhemos alguns parâmetros para a procura
parameters = {'n_estimators':[30,50,70,90],'algorithm':['SAMME', 'SAMME.R']}

scorer = make_scorer(roc_auc_score)
grid_obj = GridSearchCV(classifier, parameters, scoring=scorer)
grid_obj = grid_obj.fit(X, y)
classifier = grid_obj.best_estimator_
classifier.fit(X, y)

print(classifier.__class__.__name__)
print('Train set results:')
y_pred_train = classifier.predict(X)
classifier_scores(y,y_pred_train)
print('')
print('Test set results:')
y_pred_test = classifier.predict(X_test)
classifier_scores(y_test,y_pred_test)
print('')
# aditional info

**GradientBoostingClassifier**

In [ ]:
classifier = GradientBoostingClassifier()

# escolhemos alguns parâmetros para a procura
parameters = {'loss':['deviance', 'exponential'],'learning_rate':[0.01,0.1,1.],'min_samples_split':[3,7,10],'max_features':['auto','sqrt','log2',None]}

scorer = make_scorer(roc_auc_score)
grid_obj = GridSearchCV(classifier, parameters, scoring=scorer)
grid_obj = grid_obj.fit(X, y)
classifier = grid_obj.best_estimator_
classifier.fit(X, y)

print(classifier.__class__.__name__)
print('Train set results:')
y_pred_train = classifier.predict(X)
classifier_scores(y,y_pred_train)
print('')
print('Test set results:')
y_pred_test = classifier.predict(X_test)
classifier_scores(y_test,y_pred_test)
print('')
# aditional info

**SGDClassifier**

In [ ]:
classifier = SGDClassifier()

# escolhemos alguns parâmetros para a procura
parameters = {'eta0':[0.0001],'loss':['hinge','log','modified_huber','squared_hinge','perceptron'],'penalty':[None,'l2','l1','elasticnet'],'max_iter':[5,10,15],'learning_rate':['constant','optimal','invscaling']}

scorer = make_scorer(roc_auc_score)
grid_obj = GridSearchCV(classifier, parameters, scoring=scorer)
grid_obj = grid_obj.fit(X, y)
classifier = grid_obj.best_estimator_
classifier.fit(X, y)

print(classifier.__class__.__name__)
print('Train set results:')
y_pred_train = classifier.predict(X)
classifier_scores(y,y_pred_train)
print('')
print('Test set results:')
y_pred_test = classifier.predict(X_test)
classifier_scores(y_test,y_pred_test)
print('')